In [3]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# De 01/24 a 07/24
hospital_e_leitos_br = pd.read_csv("dados/Leitos_2024.csv", encoding='ISO-8859-1')

# Limpeza de dados
hospital_e_leitos_pb = hospital_e_leitos_br[(hospital_e_leitos_br['MUNICIPIO'] == 'JOAO PESSOA') & (hospital_e_leitos_br['COMP'] <= 202407)]

hospital_e_leitos_pb = hospital_e_leitos_pb.drop(['REGIAO', 'UF', 'MOTIVO_DESABILITACAO', 'CO_TIPO_UNIDADE', 'NATUREZA_JURIDICA', 'NO_COMPLEMENTO'], axis=1)

hospital_e_leitos_pb[['NU_TELEFONE', 'NO_EMAIL']] = hospital_e_leitos_pb[['NU_TELEFONE', 'NO_EMAIL']].fillna('Não se aplica')

hospital_e_leitos_pb = hospital_e_leitos_pb.rename(columns={
    'COMP':'ANO_MES_COMPETENCIA',
    'CNES':'ID_CNES',
    'TP_GESTAO':'TIPO_GESTAO'
})

hospital_e_leitos_pb['TIPO_GESTAO'] = hospital_e_leitos_pb['TIPO_GESTAO'].replace({
    'M':'Municipal',
    'E':'Estadual',
    'D':'Dupla',
    'S':'Sem Gestão'
})

In [4]:
# De 01/24 a 07/24
sih_jp_2024 = pd.read_csv('dados/sih_sus_joao_pessoa_2024.csv')

# Limpeza de dados
sih_jp_2024 = sih_jp_2024.drop(['ano', 'mes', 'id_municipio_paciente', 'id_gestor', 'id_aih', 'id_prestador_servico', 'cbo_2002_profissional', 'id_cid_secundario', 'id_cid_secundario_subcategoria', 'nota_fiscal', 'indicador_uf_hospital', 'indicador_uf_paciente', 'indicador_id_aih', 'tipo_financiamento_ato_profissional', 'tipo_subtipo_financiamento_ato_profissional', 'tipo_documento_pf', 'tipo_documento_pj', 'tipo_valor', 'sequencia', 'nome_remessa'], axis=1)

sih_jp_2024[['id_cid_principal', 'id_cid_principal_subcategoria']] = sih_jp_2024[['id_cid_principal', 'id_cid_principal_subcategoria']].fillna('NÃO se aplica')

C:\Users\Escritorio\AppData\Local\Temp\ipykernel_15932\3935858640.py:2: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  sih_jp_2024 = pd.read_csv('dados/sih_sus_joao_pessoa_2024.csv')
